In [6]:
import pandas as pd
import requests


In [ ]:

df = pd.read_csv("E:\Research_umich\Professor_He\IU_hackthon\input_file\hpo-kidney-genes-to-phenotype-edges.csv")

<>:2: SyntaxWarning: invalid escape sequence '\R'
<>:2: SyntaxWarning: invalid escape sequence '\R'
C:\Users\ruope\AppData\Local\Temp\ipykernel_18796\1272922246.py:2: SyntaxWarning: invalid escape sequence '\R'
  df = pd.read_csv("E:\Research_umich\Professor_He\IU_hackthon\input_file\hpo-kidney-genes-to-phenotype-edges.csv")


In [28]:
df 

,subject,predicate,object,source
0,http://identifiers.org/hgnc/124,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0005576,https://purl.humanatlas.io/vocab/hp
1,http://identifiers.org/hgnc/124,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0012622,https://purl.humanatlas.io/vocab/hp
2,http://identifiers.org/hgnc/124,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0000790,https://purl.humanatlas.io/vocab/hp
3,http://identifiers.org/hgnc/132,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0000126,https://purl.humanatlas.io/vocab/hp
4,http://identifiers.org/hgnc/144,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0000126,https://purl.humanatlas.io/vocab/hp
...,...,...,...,...
5069,http://identifiers.org/hgnc/43984,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0001919,https://purl.humanatlas.io/vocab/hp
5070,http://identifiers.org/hgnc/49094,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0100820,https://purl.humanatlas.io/vocab/hp
5071,http://identifiers.org/hgnc/49094,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0000083,https://purl.humanatlas.io/vocab/hp
5072,http://identifiers.org/hgnc/49094,https://purl.humanatlas.io/vocab/hp#has_modifier,http://purl.obolibrary.org/obo/HP_0000099,https://purl.humanatlas.io/vocab/hp


In [7]:
gene_order = [
    'CCL21', 'NRXN1', 'SLC9A3', 'SPP1', 'SERPINE2',
    "IL18", "NBAS", "SLC9A3", "TIMP2", "IGFBP7", 
    "SUGP1", "APOA1", "MMP9", "ITGAM", "FEV", 
    "TFF3", "B2M", "S100A6", "IL10", 'IL6', 
    "CXCL8", "LCN2", "HAVCR1", "CST3", "CCL2", "FABP1"
]

In [ ]:
def get_hgnc_info(gene):
    """Query OLS for HGNC term of a gene symbol."""
    url = "https://www.ebi.ac.uk/ols/api/search"
    params = {"q": gene, "ontology": "hgnc", "exact": "true"}
    r = requests.get(url, params=params)

    if r.status_code == 200:
        data = r.json()
        if data['response']['numFound'] > 0:
            hit = data['response']['docs'][0]
            iri_raw=hit.get("iri")
            iri_formal="http://identifiers.org/hgnc/"+iri_raw.rsplit("/", 1)[-1]
            return {
                "gene": gene,
                "iri":  iri_formal,
                "label": hit.get("label"),
                "source": "HGNC"
            }
    return {"gene": gene, "iri": None, "label": None, "source": None}

In [37]:
spp1=get_hgnc_info("SPP1")

In [40]:
spp1['iri']

'http://identifiers.org/hgnc/11255'

In [43]:

df_edge = pd.DataFrame(columns=["subject", "predicate","object", "source"])



In [44]:
for gene in gene_order:
    iri=get_hgnc_info(gene)
    df_edge = pd.concat([df_edge, pd.DataFrame([{
    "subject": iri['iri'],
    "predicate": "https://purl.humanatlas.io/vocab/hp#associated_with",
    "object": "http://purl.obolibrary.org/obo/HP_0001919",
    "source": "https://purl.humanatlas.io/vocab/hp"
    }])], ignore_index=True)

In [46]:
df_edge.to_csv("AMIA_gene_to_phenotype.csv")

In [47]:
df_node = pd.DataFrame(columns=["iri", "label","type", "source"])
for gene in gene_order:
    iri=get_hgnc_info(gene)
    df_node = pd.concat([df_node, pd.DataFrame([{
    "iri": iri['iri'],
    "label": gene,
    "type": "http://purl.bioontology.org/ontology/HGNC/gene	",
    "source": "https://purl.humanatlas.io/vocab/hp"
    }])], ignore_index=True)

In [48]:
df_edge.to_csv("AMIA_gene_to_phenotype_nodes_file.csv")